<center><h1>Tarea 8: Encoder-Decoder</h1></center>
<center><h4>Martínez Ostoa Néstor Iván</h4></center>
<center><h4>Aprendizaje Profundo</h4></center>
<center><h4><i>LCD - IIMAS - UNAM</i></h4></center>
<center><h4>28 de marzo del 2022</h4></center>

---

Traducción automática con una red Encoder-Decoder

## Bibilotecas a usar

In [13]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import random
import spacy
import pandas as pd

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator, TabularDataset
from torchtext.data.metrics import bleu_score

from sklearn.model_selection import train_test_split

## Conjuntos de datos

Repositorio con los datos utilizados en este notebook: 
- [Train Data](https://github.com/nestorivanmo/datasets/blob/master/tab-delimited-bilingual-sentence-pairs/train.csv)
- [Validation Data](https://github.com/nestorivanmo/datasets/blob/master/tab-delimited-bilingual-sentence-pairs/validation.csv)
- [Test Data](https://github.com/nestorivanmo/datasets/blob/master/tab-delimited-bilingual-sentence-pairs/test.csv)

In [14]:
!python -m spacy download en --quiet
!python -m spacy download es --quiet

In [15]:
spacy_english = spacy.load("en_core_web_sm")
spacy_spanish = spacy.load("es_core_news_sm")

In [16]:
def tokenize_spanish(text):
  return [token.text for token in spacy_spanish.tokenizer(text)]

def tokenize_english(text):
  return [token.text for token in spacy_english.tokenizer(text)]

spanish = Field(
    tokenize=tokenize_spanish,
    lower=True,
    init_token="<sos>",
    eos_token="<eos>"
)

english = Field(
    tokenize=tokenize_english,
    lower=True,
    init_token="<sos>",
    eos_token="<eos>"
)

fields = [('trg', english), ('src', spanish)]

In [17]:
train_data, valid_data, test_data = TabularDataset.splits(
    path='../input/mt-datasets',
    train='train.csv',
    validation='validation.csv',
    test='test.csv',
    format='csv',
    fields=fields
)

In [18]:
print(f"Registros datos de entrenamiento: {len(train_data)}")
print(f"Registros datos de validacion: {len(valid_data)}")
print(f"Registros datos de prueba: {len(test_data)}")

In [19]:
for e in train_data[:4]:
  print(e.src, '\n', e.trg)
  print()

In [20]:
spanish.build_vocab(train_data, max_size=10000, min_freq=3)
english.build_vocab(train_data, max_size=10000, min_freq=3)

In [21]:
print(f"Tokens únicos en el vocabulario del idioma alemán: {len(spanish.vocab):,}")
print(f"Tokens únicos en el vocabulario del idioma inglés: {len(english.vocab):,}")

## Creación de iteradores

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                      batch_size = BATCH_SIZE, 
                                                                      sort_within_batch=True,
                                                                      sort_key=lambda x: len(x.src),
                                                                      device = device)

In [23]:
print (f"{len(train_iterator.dataset):,} muestras en el conjunto de entrenamiento")
print (f"{len(valid_iterator.dataset):,} muestras en el conjunto de validación")
print (f"{len(test_iterator.dataset):,} muestras en el conjunto de prueba\n")

print (f"{len(train_iterator):,} batches de tamaño {BATCH_SIZE} en el conjunto de entrenamiento")
print (f"{len(valid_iterator):,} batches de tamaño {BATCH_SIZE} en el conjunto de validación")
print (f"{len(test_iterator):,} batches de tamaño {BATCH_SIZE} en el conjunto de prueba")

In [24]:
sent_len_ger, sent_len_eng = [], []
for i, data in enumerate(train_data): 
    sent_len_ger.append(len(data.src))
    sent_len_eng.append(len(data.trg))
    if i < 10 :
        print(f"Español: {' '.join(data.src)} Length: {len(data.src)}")
        print(f"Inglés: {' '.join(data.trg)} Length: {len(data.trg)}\n")

print(f"Oración con longitud máxima y mínima en alemán:\t{max(sent_len_ger)}\t{min(sent_len_ger)}")
print(f"Oración con longitud máxima y mínima en inglés:\t{max(sent_len_eng)}\t{min(sent_len_eng)}")

## Definir el modelo Encoder-Decoder
En principio se define el encoder, que es quien recibe las entradas. En este caso las entradas corresponden al idioma Alemán.

### Encoder

In [25]:
# definir el encoder
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()

        # Size of the one hot vectors that will be the input to the encoder
        self.input_size = input_size

        # Output size of the word embedding NN
        self.embedding_size = embedding_size

        # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
        self.hidden_size = hidden_size

        # Number of layers in the lstm
        self.num_layers = num_layers

        # Regularization parameter
        self.dropout = nn.Dropout(p)
        self.tag = True

        # Shape --------------------> (5376, 300) [input size, embedding dims]
        self.embedding = nn.Embedding(self.input_size, self.embedding_size)
        
        # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
        self.LSTM = nn.LSTM(self.embedding_size, hidden_size, num_layers, dropout = p)

    # Shape of x (26, 32) [Sequence_length, batch_size]
    def forward(self, x):
        # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
        embedding = self.dropout(self.embedding(x))
        
        # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
        # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
        outputs, (hidden_state, cell_state) = self.LSTM(embedding)

        return hidden_state, cell_state

In [26]:
# instanciar el encoder
input_size_encoder = len(spanish.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = float(0.5)

encoder = Encoder(
    input_size_encoder, 
    encoder_embedding_size,
    hidden_size, 
    num_layers, 
    encoder_dropout
).to(device)

print(encoder)

### Decoder

In [27]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
        super(Decoder, self).__init__()

        # Size of the one hot vectors that will be the input to the encoder
        self.input_size = input_size

        # Output size of the word embedding NN
        self.embedding_size = embedding_size

        # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
        self.hidden_size = hidden_size

        # Number of layers in the lstm
        self.num_layers = num_layers

        # Size of the one hot vectors that will be the output to the encoder (English Vocab Size)
        self.output_size = output_size

        # Regularization parameter
        self.dropout = nn.Dropout(p)
        self.tag = True

        # Shape --------------------> (5376, 300) [input size, embedding dims]
        self.embedding = nn.Embedding(self.input_size, self.embedding_size)

        # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
        self.LSTM = nn.LSTM(self.embedding_size, hidden_size, num_layers, dropout = p)

        # Shape -----------> (1024, 4556) [embedding dims, hidden size, num layers]
        self.fc = nn.Linear(self.hidden_size, self.output_size)

    # Shape of x (32) [batch_size]
    def forward(self, x, hidden_state, cell_state):

        # Shape of x (1, 32) [1, batch_size]
        x = x.unsqueeze(0)

        # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
        embedding = self.dropout(self.embedding(x))

        # Shape --> outputs (1, 32, 1024) [1, batch_size , hidden_size]
        # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
        outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

        # Shape --> predictions (1, 32, 4556) [ 1, batch_size , output_size]
        predictions = self.fc(outputs)

        # Shape --> predictions (32, 4556) [batch_size , output_size]
        predictions = predictions.squeeze(0)

        return predictions, hidden_state, cell_state

In [28]:
# instanciar el decoder
input_size_decoder = len(english.vocab) 
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
decoder_dropout = float(0.5)
output_size = len(english.vocab)

decoder = Decoder(
    input_size_decoder, 
    decoder_embedding_size,
    hidden_size, num_layers, 
    decoder_dropout, output_size
).to(device)

print(decoder)

## Definir el modelo Seq2Seq


In [29]:
# definir el modelo seq2seq que contiene al encoder y decoder
class Seq2Seq(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(Seq2Seq, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder

    def forward(self, source, target, tfr=0.5):
        # Shape - Source : (10, 32) [(Sentence length German + some padding), Number of Sentences]
        batch_size = source.shape[1]

        # Shape - Source : (14, 32) [(Sentence length English + some padding), Number of Sentences]
        target_len = target.shape[0]
        target_vocab_size = len(english.vocab)
        
        # Shape --> outputs (14, 32, 5766) 
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
        hidden_state_encoder, cell_state_encoder = self.Encoder(source)

        # Shape of x (32 elements)
        x = target[0] # Trigger token <SOS>

        for i in range(1, target_len):
            # Shape --> output (32, 5766) 
            output, hidden_state_decoder, cell_state_decoder = self.Decoder(x, hidden_state_encoder, cell_state_encoder)
            outputs[i] = output
            best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
            x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

        # Shape --> outputs (14, 32, 5766) 
        return outputs

In [30]:
# instanciar el modelo principal (seq2seq)
model = Seq2Seq(encoder, decoder)
print(model)

## Entrenamiento

### Hiperparámetros

In [31]:
# Hyperparámetros
learning_rate = 0.001
step = 0
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
pad_idx = english.vocab.stoi["<pad>"] # obtener id del relleno
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx) # omitir calcular elementos con relleno en el backpropagation

In [39]:
def translate_sentence(model, sentence, spanish, english, device, max_length=50):
    spacy_ger = spacy.load("es_core_news_sm")

    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, spanish.init_token)
    tokens.append(spanish.eos_token)
    text_to_indices = [spanish.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder(sentence_tensor)

    outputs = [english.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

In [33]:
def bleu(data, model, spanish, english, device):
    """Método para medir el rendimiento del modelo"""
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, spanish, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

In [37]:
def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss):
    print('saving')
    print()
    state = {'model': model,'best_loss': best_loss,'epoch': epoch,'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict(),}
    torch.save(state, 'checkpoint-NMT.ckpt')
    torch.save(model.state_dict(),'checkpoint-NMT-SD.ckpt')

In [35]:
def epoch_time(start_time, end_time):
    """Obtener el tiempo en minutos y segundos"""
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [40]:
import time

best_valid_loss = float('inf')
num_epochs = 50
# best_loss = 999999
# best_epoch = -1
sentence1 = "esta es mi primera oracion en español"
epoch_loss_list = []
ts1 = []

start_total_time = time.time() # inicializar 
for epoch in range(num_epochs):
    epoch_loss = 0.0
    # calcular el tiempo que tarda cada epoca
    start_time = time.time() # inicializar

    model.train(True)
    for batch_idx, batch in enumerate(train_iterator):
        input = batch.src.to(device)
        target = batch.trg.to(device)

        # Pass the input and target for model's forward method
        output = model(input, target)
        output = output[1:].reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        # Clear the accumulating gradients
        optimizer.zero_grad()

        # Calculate the loss value for every epoch
        loss = criterion(output, target)

        # Calculate the gradients for weights & biases using back-propagation
        loss.backward()

        # Clip the gradient value is it exceeds > 1
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Update the weights values using the gradients we calculated using bp 
        optimizer.step()
        step += 1
        epoch_loss += loss.item()
        # writer.add_scalar("Training loss", loss, global_step=step)

    # if epoch_loss < best_loss:
    #   best_loss = epoch_loss
    #   best_epoch = epoch
    #   checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
    #   if ((epoch - best_epoch) >= 10):
    #     print("no improvement in 10 epochs, break")
    #     break

    if epoch_loss < best_valid_loss: # guardar el mejor modelo
        best_valid_loss = epoch_loss
        checkpoint_and_save(model, best_valid_loss, epoch, optimizer, epoch_loss) 
      
    tmp_loss = epoch_loss / len(train_iterator)
    epoch_loss_list.append(tmp_loss)
    
    end_time = time.time() # obtener el tiempo final
    # obtener los minutos y segundos del tiempo de ejecución
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # ----------------------------------------------------------------------------------
    # evaluación
    model.eval()
    with torch.no_grad():
        translated_sentence1 = translate_sentence(model, sentence1, spanish, english, device, max_length=50)
        print(f"Translated example sentence 1:\n{' '.join(translated_sentence1)}")
        ts1.append(translated_sentence1)
    # ----------------------------------------------------------------------------------

    print(f"Epoch - {epoch+1} / {num_epochs} | Epoch Time:\t{epoch_mins}m {epoch_secs}s | Epoch_Loss:\t{tmp_loss:.3f}")
    print()
  
print(epoch_loss / len(train_iterator))

score = bleu(test_data[1:100], model, spanish, english, device)
print(f"Bleu score {score*100:.2f}")

end_total_time = time.time() # obtener el tiempo final
epoch_mins, epoch_secs = epoch_time(start_total_time, end_total_time)
print (f"\nTotal time for training model:\t{epoch_mins}m {epoch_secs}s")

<pre>
saving

Translated example sentence 1:
a blue a a a a game of a group of a swing thin school gyro touch heels short awaits demonstrates wheelchairs wires expanse oxford loading oxford removing tethered vegetation sunrise multicolor cardigan janitor tomato cubicle projected secured snap surgery amid fancy bites apart attentively down challenge religious things crate
Epoch - 1 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.926

saving

Translated example sentence 1:
a blue in a a a game of red striped hardwood discuss yelling tulips pause waited ally happening slam slide passersby sweatshirt violinist pack crotch rack backhoe propane attaches asking texas dunk turbans aqua tye tye ripped laundromat law law bunny landed procession sheepdog solitary 30 bagpipe law law straining
Epoch - 2 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.891

saving

Translated example sentence 1:
a blue is on a a down some a baseball white five used used zone purse twirl mustachioed she can contraption mad mad weathered excitement heavyset gyro bookstore janitor law classic law scissors bookstore cars wakeboard janitor law law enforcement law law pajamas miami junk metallic reach onlooker crab three
Epoch - 3 / 30 | Epoch Time:	3m 30s | Epoch_Loss:	1.859

saving

Translated example sentence 1:
a window a a a small a small a small a girls smiles force french constructed honor related participants aerial participants california stay healthcare fog fog fog graying judges major streams how diverse box shop trash rollerblader rollerblader elbow dozen twirls rollerblader solitary wicker forested shelter stall mountaintop scenic act
Epoch - 4 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.829

saving

Translated example sentence 1:
a blue on a a a small <unk> a laugh group of a good length climb m rod dragon parts mustard return shaded handbag electrician cycles briefcase topped damaged rainstorm solitary claus beige hills preparation teal bearer mustard years shaded outfits spectacular shaded projected pitched metropolitan shaded create fills pokes
Epoch - 5 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.797

saving

Translated example sentence 1:
a blue a a group of worker sitting road abandoned moving vietnamese personnel breaking congregating dumpster college pub seemingly paneled monkey honor gyro features studies ohio strikes equipped butchering competes on a a good picture of straw stone figure points dough with resting himself with tasty bones features sprinkled excitedly
Epoch - 6 / 30 | Epoch Time:	3m 28s | Epoch_Loss:	1.772

saving

Translated example sentence 1:
a blue a a a football of young men carve cord technique memorial bears squat features bears lounge flames waring bears paneled doo motorboat tye sprays air track . <eos>
Epoch - 7 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.750

saving

Translated example sentence 1:
a blue . <eos>
Epoch - 8 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.732

saving

Translated example sentence 1:
a blue in a a group of smoking a red boat <eos>
Epoch - 9 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.705

saving

Translated example sentence 1:
a blue in the <eos>
Epoch - 10 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.694

saving

Translated example sentence 1:
a blue in a a large crowd this ready picture picture of a cement baggage road children pain fists pain veteran pain pain veteran unique toe drainage used cry chart where surfers elevator elevator primitive primitive honor toss features unfinished cabin socks opens salad walker stride mark butt crying barber
Epoch - 11 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.656

saving

Translated example sentence 1:
a blue on the inside building . <eos>
Epoch - 12 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.639

saving

Translated example sentence 1:
a blue in a group of enjoys dirty displays design tool tool . <eos>
Epoch - 13 / 30 | Epoch Time:	3m 28s | Epoch_Loss:	1.629

saving

Translated example sentence 1:
a blue standing on on a the frisbee . <eos>
Epoch - 14 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.585

saving

Translated example sentence 1:
a blue on the counter belts portable ticket jumpsuits sprinkled jogger beagle toyota karaoke bowler metropolitan corral rollerblader rollerblader dozens sprinkles forrest aerial left . <eos>
Epoch - 15 / 30 | Epoch Time:	3m 28s | Epoch_Loss:	1.575

saving

Translated example sentence 1:
a blue standing on on a a a small crowd <eos>
Epoch - 16 / 30 | Epoch Time:	3m 28s | Epoch_Loss:	1.560

saving

Translated example sentence 1:
a blue blue standing in on horses three animal camouflage horns corral scantily heavyset frozen defends jack find bulls angry fit short business business wear decorations laundromat safe suspended up belts obama tasty flowery hikes pony pony latino jet schoolgirl law operated ways stacking views an starring fork instructor rainstorm
Epoch - 17 / 30 | Epoch Time:	3m 28s | Epoch_Loss:	1.539

saving

Translated example sentence 1:
a blue in the water with aprons panda observing microscopes shooting a lap shot of gets bread help cup pot board . <eos>
Epoch - 18 / 30 | Epoch Time:	3m 28s | Epoch_Loss:	1.535

saving

Translated example sentence 1:
a window . <eos>
Epoch - 19 / 30 | Epoch Time:	3m 28s | Epoch_Loss:	1.522

saving

Translated example sentence 1:
a blue standing on a an three leg picture ready into panda depicting pigtails under clutching carrying still food a girl who yellow snaps mcdonalds waterskier law law waterskier waterskier law law warehouse maneuver firetrucks painters painters texas healthcare better better dozen crouches priest lets sooners personnel chart begging paying
Epoch - 20 / 30 | Epoch Time:	3m 28s | Epoch_Loss:	1.490

Translated example sentence 1:
a blue while one congregate gliding signing reflective vests butchering names names tutu barriers mustard visible follow grabbing yelling island zebra tattered attaches bananas field . tan range art sits karate pose pose cross painter painter dugout law braces toss ragged ref dozens texts tiles elevator toss troops crayon jumpsuits
Epoch - 21 / 30 | Epoch Time:	3m 26s | Epoch_Loss:	1.495

saving

Translated example sentence 1:
a blue standing on <eos>
Epoch - 22 / 30 | Epoch Time:	3m 28s | Epoch_Loss:	1.479

saving

Translated example sentence 1:
a blue while on displays some blown draped place in some long feet foreign studio lines leaves . <eos>
Epoch - 23 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.448

Translated example sentence 1:
a blue standing on <eos>
Epoch - 24 / 30 | Epoch Time:	3m 26s | Epoch_Loss:	1.448

saving

Translated example sentence 1:
a blue blue on the grass pool <eos>
Epoch - 25 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.437

saving

Translated example sentence 1:
a blue standing on the an three caught japanese suspended 6 descent lets using laptops . <eos>
Epoch - 26 / 30 | Epoch Time:	3m 28s | Epoch_Loss:	1.413

saving

Translated example sentence 1:
a blue while on a mountain glass room bull outdoors . <eos>
Epoch - 27 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.398

Translated example sentence 1:
a blue is standing on <eos>
Epoch - 28 / 30 | Epoch Time:	3m 27s | Epoch_Loss:	1.401

saving

Translated example sentence 1:
a blue on the bar military lights cigar leaving unique bamboo <unk> fountains garden . <eos>
Epoch - 29 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.393

saving

Translated example sentence 1:
a blue standing on <eos>
Epoch - 30 / 30 | Epoch Time:	3m 29s | Epoch_Loss:	1.371

1.3713945452834437
Bleu score 1.87

Total time for training model:	107m 43s
</pre>

<pre>
saving

Translated example sentence 1:
a a a man . <eos>
Epoch - 1 / 20 | Epoch Time:	3m 29s | Epoch_Loss:	4.719

saving

Translated example sentence 1:
a man in a a a a . <eos>
Epoch - 2 / 20 | Epoch Time:	3m 29s | Epoch_Loss:	3.985

saving

Translated example sentence 1:
a man . <eos>
Epoch - 3 / 20 | Epoch Time:	3m 29s | Epoch_Loss:	3.576

saving

Translated example sentence 1:
a man in a a . <eos>
Epoch - 4 / 20 | Epoch Time:	3m 29s | Epoch_Loss:	3.333

saving

Translated example sentence 1:
a man in a a a a a a a a a a . <eos>
Epoch - 5 / 20 | Epoch Time:	3m 30s | Epoch_Loss:	3.132

saving

Translated example sentence 1:
a man in a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
Epoch - 6 / 20 | Epoch Time:	3m 31s | Epoch_Loss:	2.965

saving

Translated example sentence 1:
a blue man . <eos>
Epoch - 7 / 20 | Epoch Time:	3m 30s | Epoch_Loss:	2.848

saving

Translated example sentence 1:
a man in a a a a a a a a a a a young a small . <eos>
Epoch - 8 / 20 | Epoch Time:	3m 31s | Epoch_Loss:	2.721

saving

Translated example sentence 1:
a man in a a a a a a a a a a a a a a a a a a a a a a a a a a a a a child <eos>
Epoch - 9 / 20 | Epoch Time:	3m 29s | Epoch_Loss:	2.605

saving

Translated example sentence 1:
a man in a a a a . <eos>
Epoch - 10 / 20 | Epoch Time:	3m 29s | Epoch_Loss:	2.525

saving

Translated example sentence 1:
a blue man <eos>
Epoch - 11 / 20 | Epoch Time:	3m 29s | Epoch_Loss:	2.432

saving

Translated example sentence 1:
a man in a a a <unk> a <unk> a <unk> a . <eos>
Epoch - 12 / 20 | Epoch Time:	3m 28s | Epoch_Loss:	2.354

saving

Translated example sentence 1:
a blue a a a blue <eos>
Epoch - 13 / 20 | Epoch Time:	3m 29s | Epoch_Loss:	2.286

saving

Translated example sentence 1:
a blue in a a a a a a a a a a a a a a large crowd of a a a red <unk> is hanging in the side the side of a a large group of a red of water <eos>
Epoch - 14 / 20 | Epoch Time:	3m 29s | Epoch_Loss:	2.230

saving

Translated example sentence 1:
a blue in a a a a a a a a a a soccer of a a <unk> a a a <unk> white race is taking a a a a race a <unk> a <unk> piece of a jumper burger sitting in the real troops real reclines sits in side
Epoch - 15 / 20 | Epoch Time:	3m 28s | Epoch_Loss:	2.188

saving

Translated example sentence 1:
a blue a man is <eos>
Epoch - 16 / 20 | Epoch Time:	3m 28s | Epoch_Loss:	2.139

saving

Translated example sentence 1:
a blue in a a a small <unk> a <unk> a giant crowd of bright white aim handicapped short stripes m softball pose . <eos>
Epoch - 17 / 20 | Epoch Time:	3m 29s | Epoch_Loss:	2.075

saving

Translated example sentence 1:
a blue blue a a <unk> <eos>
Epoch - 18 / 20 | Epoch Time:	3m 28s | Epoch_Loss:	2.033

saving

Translated example sentence 1:
a blue a blue a race of a football race of a football . <eos>
Epoch - 19 / 20 | Epoch Time:	3m 28s | Epoch_Loss:	1.991

saving

Translated example sentence 1:
a blue a blue <unk> looking like a a small <unk> stand in the <unk> striped top bathing vibrant soda pillar boys painters navigating rack last moose grasping lively save shadowed pro bush shaded winter skirt wicker slightly muscular statute special aim posters thumb begins chubby crystal plane dunes .
Epoch - 20 / 20 | Epoch Time:	3m 28s | Epoch_Loss:	1.962

1.961763057056835
Bleu score 2.50

Total time for training model:	72m 28s
</pre>

In [42]:
# graficar las perdidas
import matplotlib.pyplot as plt

print (f"Epochs: {len(epoch_loss_list)}")

# Graficar accuracy y loss
fig, ax = plt.subplots(figsize=(15,10))
plt.plot(epoch_loss_list, label='Training loss', marker='o', color='orange')
plt.title('Losses', fontsize=15)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid()
plt.legend()
plt.savefig('loss.png')
plt.show()

### Descargar el mejor modelo entrenado

In [45]:
checkpoint = 'checkpoint-NMT-SD.ckpt'

# descargar el mejor modelo
files.download(checkpoint)

# descargar gráfica de perdidas
files.download('loss.png')